In [2]:
import h5py
import numpy as np
from dtw import dtw
from tqdm import trange

/home/sshimizu/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
DATAPATH = '../../brain_p/data/pca_fMRI_per_cm'
WRITEPATH = '../../brain_p/data/dtw'

In [1]:
# 連番からevset, evnoをつくる
def make_ev(num):
    counter = 0
    while((num + 1) % 30 != 0):
        num += 1
        counter += 1
    evset = int((num + 1) / 30)
    evno = int(30 - counter)
    return str(evset).zfill(2) + "_" + str(evno).zfill(2)
make_ev(370)

'13_11'

In [5]:
# 類似度の行列dataを360*360の隣接行列に変換する
def array_to_adjacency_matrix(data):
    N = 360
    # dataを分割する
    keep = 0
    cut = N - 1
    ary = []
    while(cut > 0):
        ary.append(data[keep : keep + cut])
        keep += cut
        cut = cut - 1
    ary = np.array(ary)
    # 上三角行列に入れてから隣接行列をつくる
    mat = []
    for i in range(N-1):
        ary_i = np.append(np.zeros(i+1), ary[i])
        mat.append(ary_i)
    mat.append(np.zeros(N))
    mat = np.array(mat)
    mat = mat + mat.T
    return mat

In [ ]:
# for person in range(27):
#     hdfpath = DATAPATH + "/01" + str(person+1).zfill(2) + ".hdf"
#     writepath = WRITEPATH + "/01" + str(person+1).zfill(2) + ".hdf"
hdfpath = DATAPATH + "/0101.hdf"
writepath = WRITEPATH + "/0101.hdf"

with h5py.File(hdfpath,'r') as read_file:
    similarity_per_person = []
    for i in range(360):
        for j in range(i+1, 360):
            similarity_per_cm_combi = []
            for k in range(1000):
                x = read_file[make_ev(i)].value
                y = read_file[make_ev(j)].value
                l2_norm = lambda x, y: (x - y) ** 2
                d, cost_matrix, acc_cost_matrix, path = dtw(x[:,k].reshape(-1, 1), y[:,k].reshape(-1, 1), dist=l2_norm)
                similarity_per_cm_combi.append(d)
            mean_per_combi = np.mean(similarity_per_cm_combi)
            similarity_per_person.append(mean_per_combi)
    similarity_per_person = np.array(similarity_per_person)
    adjacency_matrix = array_to_adjacency_matrix(similarity_per_person)
with h5py.File(writepath, 'w') as outfile:
    outfile.create_dataset('adjacency_matrix', data = adjacency_matrix)